train_model.py

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train/255.0
x_test = x_test/255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))
model.evaluate(x_test, y_test)
model.save('cnn_mnist_model.h5')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 61s 31ms/step - accuracy: 0.8904 - loss: 0.3555 - val_accuracy: 0.9714 - val_loss: 0.0899
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 102s 42ms/step - accuracy: 0.9827 - loss: 0.0551 - val_accuracy: 0.9853 - val_loss: 0.0480
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 72s 37ms/step - accuracy: 0.9883 - loss: 0.0372 - val_accuracy: 0.9874 - val_loss: 0.0385
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 79s 35ms/step - accuracy: 0.9907 - loss: 0.0298 - val_accuracy: 0.9901 - val_loss: 0.0281
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 61s 32ms/step - accuracy: 0.9933 - loss: 0.0215 - val_accuracy: 0.9899 - val_loss: 0.0297
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9870 - loss: 0.0368


convert_tflite.py

In [4]:
import tensorflow as tf

model = tf.keras.models.load_model('cnn_mnist_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('cnn_mnist_model.tflite', 'wb') as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpfug13gi0'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  138084078919248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138084078916752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138084078914832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138084078918480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138084078920208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138084078920592: TensorSpec(shape=(), dtype=tf.resource, name=None)


run_inference.py

In [6]:
import tensorflow as tf
import numpy as np
import time
import cv2

interpreter = tf.lite.Interpreter(model_path='cnn_mnist_model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    img = np.expand_dims(img, axis=-1)
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0
    return img

image_path = 'test_image.png'
input_data = preprocess_image(image_path)

interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
predicted_class = np.argmax(output_data)
print('Predicted class:', predicted_class)

times = []
for i in range(20):
    start_time = time.time()
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    delta = time.time() - start_time
    times.append(delta)

avg_time = np.mean(times)
fps = 1 / avg_time
print(f"Average time per inference: {avg_time:.4f} seconds, FPS: {fps:.2f}")

Predicted class: 5
Average time per inference: 0.0001 seconds, FPS: 7581.89
